# Aggregating and joining data

This is the second introductory tutorial to Ibis. If you are new to Ibis, you may want to start
by the first tutorial, _01-Introduction-to-Ibis_.

In the first tutorial, we saw how to operate on the data of a table. We will work again with
the `countries` table as we did previously.

In [1]:
import os
import ibis


ibis.options.interactive = True

connection = ibis.sqlite.connect(os.path.join('data', 'geography.db'))
countries = connection.table('countries')

countries['name', 'continent', 'area_km2', 'population']

                     name continent   area_km2  population
0                 Andorra        EU      468.0       84000
1    United Arab Emirates        AS    82880.0     4975593
2             Afghanistan        AS   647500.0    29121286
3     Antigua and Barbuda        NA      443.0       86754
4                Anguilla        NA      102.0       13254
..                    ...       ...        ...         ...
247                 Yemen        AS   527970.0    23495361
248               Mayotte        AF      374.0      159042
249          South Africa        AF  1219912.0    49000000
250                Zambia        AF   752614.0    13460305
251              Zimbabwe        AF   390580.0    13061000

[252 rows x 4 columns]

## Expressions

We will continue by exploring the data by continent. We will start by creating an expression
with the continent names, since our table only contains the abbreviations.

An expression is one or more operations performed over the data. They can be used to retrieve the
data or to build more complex operations.

In this case we will use a `case` conditional statement to replace values depending on a condition.
A `case` expression will return a case builder, and must be followed by one or more `when` calls,
optionally an `else_` call, and must end with a call to `end`, to complete the full expression.
The expression where `case` is called (`countries['continent']` in this case)
is evaluated to see if it's equal to any of the first arguments of the calls to `when`. And the second
argument is returned. If the value does not match any of the `when` values, the value of `else_` is returned.

In [2]:
continent_name = (countries['continent'].case()
                                        .when('NA', 'North America')
                                        .when('SA', 'South America')
                                        .when('EU', 'Europe')
                                        .when('AF', 'Africa')
                                        .when('AS', 'Asia')
                                        .when('OC', 'Oceania')
                                        .when('AN', 'Anctartica')
                                        .else_('Unknown continent')
                                        .end()
                                        .name('continent_name'))
continent_name

0             Europe
1               Asia
2               Asia
3      North America
4      North America
           ...      
247             Asia
248           Africa
249           Africa
250           Africa
251           Africa
Name: tmp, Length: 252, dtype: object

What we did is take the values of the column `countries['continent']`, and we created a calculated
column with the names of the continents, as defined in the `when` methods.

This calculated column is an expression. The computations didn't happen when defining the `continent_name`
variable, and the results are not stored. They have been computed when we printed its content.

We can see that by checking the type of `continent_name`:

In [3]:
type(continent_name)

ibis.expr.types.StringColumn

In the next tutorial we will see more about eager and lazy mode, and when operations are being
executed. For now we can think that the query to the database happens only when we want to see
the results.

The important part is that now we can use our `continent_name` expression in other expressions.
For example, since this is a column (a `StringColumn` to be specific), we can use it as a column
to query the countries table.

Note that when we created the expression we added `.name('continent_name')` to it, so the column
has a name when being returned.

In [4]:
countries['name', continent_name, 'area_km2', 'population']

                     name continent_name   area_km2  population
0                 Andorra         Europe      468.0       84000
1    United Arab Emirates           Asia    82880.0     4975593
2             Afghanistan           Asia   647500.0    29121286
3     Antigua and Barbuda  North America      443.0       86754
4                Anguilla  North America      102.0       13254
..                    ...            ...        ...         ...
247                 Yemen           Asia   527970.0    23495361
248               Mayotte         Africa      374.0      159042
249          South Africa         Africa  1219912.0    49000000
250                Zambia         Africa   752614.0    13460305
251              Zimbabwe         Africa   390580.0    13061000

[252 rows x 4 columns]

Just for illustration, let's repeat the same query, but renaming the expression to `continent`
when using it in the list of columns to fetch.

In [5]:
countries['name', continent_name.name('continent'), 'area_km2', 'population']

                     name      continent   area_km2  population
0                 Andorra         Europe      468.0       84000
1    United Arab Emirates           Asia    82880.0     4975593
2             Afghanistan           Asia   647500.0    29121286
3     Antigua and Barbuda  North America      443.0       86754
4                Anguilla  North America      102.0       13254
..                    ...            ...        ...         ...
247                 Yemen           Asia   527970.0    23495361
248               Mayotte         Africa      374.0      159042
249          South Africa         Africa  1219912.0    49000000
250                Zambia         Africa   752614.0    13460305
251              Zimbabwe         Africa   390580.0    13061000

[252 rows x 4 columns]

## Aggregating data

Now, let's group our data by continent, and let's find the total population of each.

In [6]:
countries.group_by(continent_name).aggregate(countries['population'].sum().name('total_population'))

  continent_name  total_population
0         Africa        1021238685
1     Anctartica               170
2           Asia        4130584841
3         Europe         750724554
4  North America         540204371
5        Oceania          36067549
6  South America         400143568

We can see how Asia is the most populated country, followed by Africa. Antarctica is the least populated,
as we would expect.

The code to aggregate has two main parts:
- The `group_by` method, that receive the column, expression or list of them to group by
- The `aggregate` method, that receives an expression with the reduction we want to apply

To make things a bit clearer, let's first save the reduction.

In [7]:
total_population = countries['population'].sum().name('total_population')
total_population

6878963738

As we can see, if we perform the operation directly, we will get the sum of the total in the column.

But if we take the `total_population` expression as the parameter of the `aggregate` method, then the total is computed
over every group defined by the `group_by` method.

In [8]:
countries.group_by(continent_name).aggregate(total_population)

  continent_name  total_population
0         Africa        1021238685
1     Anctartica               170
2           Asia        4130584841
3         Europe         750724554
4  North America         540204371
5        Oceania          36067549
6  South America         400143568

If we want to compute two aggregates at the same time, we can pass a list to the `aggregate` method.

For illustration, we use the `continent` column, instead of the `continent_names` expression. We can
use both column names and expressions, and also a list with any of them (e.g. `[continent_names, 'name']`.

In [9]:
countries.group_by('continent').aggregate([total_population,
                                           countries['area_km2'].mean().name('average_area')])

  continent  total_population  average_area
0        AF        1021238685  5.234534e+05
1        AN               170  2.802439e+06
2        AS        4130584841  6.196685e+05
3        EU         750724554  4.293017e+05
4        NA         540204371  5.836313e+05
5        OC          36067549  3.044157e+05
6        SA         400143568  1.272751e+06

## Joining data

Now we are going to get the total gross domestic product (GDP) for each continent. In this case, the GDP data
is not in the same table `countries`, but in a table `gdp`.

In [10]:
gdp = connection.table('gdp')
gdp

     country_code  year         value
0             ABW  1986  4.054634e+08
1             ABW  1987  4.876025e+08
2             ABW  1988  5.964236e+08
3             ABW  1989  6.953044e+08
4             ABW  1990  7.648871e+08
...           ...   ...           ...
9995          SVK  2002  3.513034e+10
9996          SVK  2003  4.681659e+10
9997          SVK  2004  5.733202e+10
9998          SVK  2005  6.278531e+10
9999          SVK  2006  7.070810e+10

[10000 rows x 3 columns]

The table contains information for different years, we can easily check the range with:

In [11]:
gdp['year'].min(), gdp['year'].max()

(1960, 2017)

We are going to just take the information about the last available year, 2017.

In [12]:
gdp_2017 = gdp.filter(gdp['year'] == 2017)['country_code', 'value']
gdp_2017

    country_code         value
0            ABW  2.700559e+09
1            AFG  2.019176e+10
2            AGO  1.221238e+11
3            ALB  1.302506e+10
4            AND  3.013387e+09
..           ...           ...
242          XKX  7.227700e+09
243          YEM  2.681870e+10
244          ZAF  3.495541e+11
245          ZMB  2.586814e+10
246          ZWE  2.281301e+10

[247 rows x 2 columns]

Now, we are going to join this data with the `countries` table so we can obtain the continent
of each country. The `countries` table has several different codes for the countries. Let's find out which
one matches the three letter code in the `gdp` table.

In [13]:
countries['iso_alpha2', 'iso_alpha3', 'iso_numeric', 'fips', 'name']

    iso_alpha2 iso_alpha3  iso_numeric fips                  name
0           AD        AND           20   AN               Andorra
1           AE        ARE          784   AE  United Arab Emirates
2           AF        AFG            4   AF           Afghanistan
3           AG        ATG           28   AC   Antigua and Barbuda
4           AI        AIA          660   AV              Anguilla
..         ...        ...          ...  ...                   ...
247         YE        YEM          887   YM                 Yemen
248         YT        MYT          175   MF               Mayotte
249         ZA        ZAF          710   SF          South Africa
250         ZM        ZMB          894   ZA                Zambia
251         ZW        ZWE          716   ZI              Zimbabwe

[252 rows x 5 columns]

The `country_code` in `gdp` corresponds to `iso_alpha2` in the `countries` table. We can also see
how the `gdp` table has `10,000` rows, while `countries` has `252`. We will start joining the
two tables by the codes that match, discarding the codes that do not exist in both tables.
This is called an inner join.

In [27]:
countries_and_gdp = countries.inner_join(gdp,
                                         predicates=countries['iso_alpha3'] == gdp['country_code'])
countries_and_gdp[countries, gdp]

     iso_alpha2 iso_alpha3  iso_numeric fips      name           capital  \
0            AD        AND           20   AN   Andorra  Andorra la Vella   
1            AD        AND           20   AN   Andorra  Andorra la Vella   
2            AD        AND           20   AN   Andorra  Andorra la Vella   
3            AD        AND           20   AN   Andorra  Andorra la Vella   
4            AD        AND           20   AN   Andorra  Andorra la Vella   
...         ...        ...          ...  ...       ...               ...   
9482         ZW        ZWE          716   ZI  Zimbabwe            Harare   
9483         ZW        ZWE          716   ZI  Zimbabwe            Harare   
9484         ZW        ZWE          716   ZI  Zimbabwe            Harare   
9485         ZW        ZWE          716   ZI  Zimbabwe            Harare   
9486         ZW        ZWE          716   ZI  Zimbabwe            Harare   

      area_km2  population continent country_code  year         value  
0        468.0 

In [29]:
print(ibis.sqlite.compile(countries_and_gdp))
print(ibis.sqlite.compile(countries_and_gdp[countries, gdp]))

SELECT t0.iso_alpha2, t0.iso_alpha3, t0.iso_numeric, t0.fips, t0.name, t0.capital, t0.area_km2, t0.population, t0.continent, t1.country_code, t1.year, t1.value 
FROM base.countries AS t0 JOIN base.gdp AS t1 ON t0.iso_alpha3 = t1.country_code
SELECT t0.iso_alpha2, t0.iso_alpha3, t0.iso_numeric, t0.fips, t0.name, t0.capital, t0.area_km2, t0.population, t0.continent, t1.country_code, t1.year, t1.value 
FROM base.countries AS t0 JOIN base.gdp AS t1 ON t0.iso_alpha3 = t1.country_code


In [14]:
countries_and_gdp = (countries.inner_join(gdp,
                                          predicates=countries['iso_alpha3'] == gdp['country_code'])
                              .filter(gdp['year'] == 2017))
expr = countries_and_gdp['country_code', 'name', 'year', 'value']
expr

     country_code     name  year         value
0             AND  Andorra  1970  7.861921e+07
1             AND  Andorra  1970  7.861921e+07
2             AND  Andorra  1970  7.861921e+07
3             AND  Andorra  1970  7.861921e+07
4             AND  Andorra  1970  7.861921e+07
...           ...      ...   ...           ...
9995          AND  Andorra  2010  3.355695e+09
9996          AND  Andorra  2010  3.355695e+09
9997          AND  Andorra  2010  3.355695e+09
9998          AND  Andorra  2010  3.355695e+09
9999          AND  Andorra  2010  3.355695e+09

[10000 rows x 4 columns]

In [22]:
expr = countries.inner_join(gdp_2017, predicates=countries['iso_alpha3'] == gdp['country_code'])
print(ibis.sqlite.compile(expr[countries, gdp_2017]))
print(expr[countries, gdp_2017])

SELECT t0.iso_alpha2, t0.iso_alpha3, t0.iso_numeric, t0.fips, t0.name, t0.capital, t0.area_km2, t0.population, t0.continent, t1.country_code, t1.value 
FROM base.countries AS t0 JOIN (SELECT t2.country_code AS country_code, t2.value AS value 
FROM (SELECT t3.country_code AS country_code, t3.year AS year, t3.value AS value 
FROM base.gdp AS t3 
WHERE t3.year = ?) AS t2) AS t1 ON t0.iso_alpha3 = base.gdp.country_code


OperationalError: (sqlite3.OperationalError) no such column: base.gdp.country_code
[SQL: SELECT t0.iso_alpha2, t0.iso_alpha3, t0.iso_numeric, t0.fips, t0.name, t0.capital, t0.area_km2, t0.population, t0.continent, t1.country_code, t1.value 
FROM base.countries AS t0 JOIN (SELECT t2.country_code AS country_code, t2.value AS value 
FROM (SELECT t3.country_code AS country_code, t3.year AS year, t3.value AS value 
FROM base.gdp AS t3 
WHERE t3.year = ?) AS t2) AS t1 ON t0.iso_alpha3 = base.gdp.country_code
 LIMIT ? OFFSET ?]
[parameters: (2017, 10000, 0)]
(Background on this error at: http://sqlalche.me/e/e3q8)

In [17]:
print(str(ibis.sqlite.compile(expr)))

SELECT t0.country_code, t0.name, t0.year, t0.value 
FROM (SELECT t1.iso_alpha2 AS iso_alpha2, t1.iso_alpha3 AS iso_alpha3, t1.iso_numeric AS iso_numeric, t1.fips AS fips, t1.name AS name, t1.capital AS capital, t1.area_km2 AS area_km2, t1.population AS population, t1.continent AS continent, t1.country_code AS country_code, t1.year AS year, t1.value AS value 
FROM (SELECT t2.iso_alpha2 AS iso_alpha2, t2.iso_alpha3 AS iso_alpha3, t2.iso_numeric AS iso_numeric, t2.fips AS fips, t2.name AS name, t2.capital AS capital, t2.area_km2 AS area_km2, t2.population AS population, t2.continent AS continent, t3.country_code AS country_code, t3.year AS year, t3.value AS value 
FROM base.countries AS t2 JOIN base.gdp AS t3 ON t2.iso_alpha3 = t3.country_code) AS t1, base.gdp 
WHERE base.gdp.year = ?) AS t0


In [16]:
countries_and_gdp = countries.inner_join(gdp_2017,
                                         predicates=countries['iso_alpha3'] == gdp_2017['country_code'])
countries_and_gdp.materialize()

OperationalError: (sqlite3.OperationalError) ambiguous column name: country_code
[SQL: SELECT t0.iso_alpha2, t0.iso_alpha3, t0.iso_numeric, t0.fips, t0.name, t0.capital, t0.area_km2, t0.population, t0.continent, t1.country_code, t1.value 
FROM base.countries AS t0 JOIN (SELECT t2.country_code AS country_code, t2.value AS value 
FROM (SELECT t3.country_code AS country_code, t3.year AS year, t3.value AS value 
FROM base.gdp AS t3 
WHERE t3.year = ?) AS t2) AS t1 ON t0.iso_alpha3 = country_code, (SELECT t3.country_code AS country_code, t3.year AS year, t3.value AS value 
FROM base.gdp AS t3 
WHERE t3.year = ?)
 LIMIT ? OFFSET ?]
[parameters: (2017, 2017, 10000, 0)]
(Background on this error at: http://sqlalche.me/e/e3q8)

We have called the `inner_join` method of the `countries` table and passed
the `gdp` table as a parameter. The method receives a second parameter, `predicates`, that is used to specify
how the join will be performed. In this case we want the `iso_alpha3` column in `countries` to
match the `country_code` column in `gdp`. This is specified with the expression
`countries['iso_alpha3'] == gdp['country_code']`.

Finally, before we can use the values of the joined query, we need to materialize the result by calling
its `materialize` method. Let's look with another example why joins need to be materialized.

We will now "clean up" the `countries` table before performing the join, by removing the columns we
don't want and renaming the code column we are going to use.

In [12]:
countries_simplified = countries[countries['iso_alpha3'].name('country_code'), 'continent']
countries_simplified

    country_code continent
0            AND        EU
1            ARE        AS
2            AFG        AS
3            ATG        NA
4            AIA        NA
..           ...       ...
247          YEM        AS
248          MYT        AF
249          ZAF        AF
250          ZMB        AF
251          ZWE        AF

[252 rows x 2 columns]

Now, we can join again as we did before, but with the the expression `countries_simplified` instead of with
the table `countries` directly.

This will generate an exception.

In [13]:
countries_and_gdp = countries_simplified.inner_join(gdp,
                                                    predicates=countries_simplified['country_code'] == gdp['country_code'])
countries_and_gdp.materialize()

RelationError: Joined tables have overlapping names: ['country_code']

An exception is happening because the result would include the column `country_code` of
both tables (with the same column name). And duplicated column names are not allowed.

What we will do is to perform the join, but we won't yet materialize it.

In [14]:
countries_and_gdp = countries_simplified.inner_join(gdp,
                                                    predicates=countries_simplified['country_code'] == gdp['country_code'])

This works, and we got the resulting expression in `countries_and_gdp`.

Next, we will select from this resulting expression the columns that we are interested in - the continent, the value
of the GDP, and only one country code.

The result does not have duplicate columns now, after this extra step, so we can materialize:

In [15]:
gdp_with_continent = countries_and_gdp[countries_simplified['country_code'],
                                       countries_simplified['continent'],
                                       gdp['year'],
                                       gdp['value'].name('gdp')].materialize()
gdp_with_continent

     country_code continent  year           gdp
0             AND        EU  1970  7.861921e+07
1             AND        EU  1971  8.940982e+07
2             AND        EU  1972  1.134082e+08
3             AND        EU  1973  1.508201e+08
4             AND        EU  1974  1.865587e+08
...           ...       ...   ...           ...
9482          ZWE        AF  2013  1.909102e+10
9483          ZWE        AF  2014  1.949552e+10
9484          ZWE        AF  2015  1.996312e+10
9485          ZWE        AF  2016  2.054868e+10
9486          ZWE        AF  2017  2.281301e+10

[9487 rows x 4 columns]

We could also rename columns before materializing. That would be useful for example
if `countries` also had a `value` column and we wanted to get both, the one in
`gdp` and the one in `countries`.

The table contains the GDP information per country, for a number of years, let's check which years.